In [ ]:
import pandas as pd
import numpy as np

window_size = 73
def estimate_local_tch_variances_multifeature(df1, df2, df3, window_size=window_size):
     # Ensure all DataFrames have the same shape and columns
    if not (df1.shape == df2.shape == df3.shape):
        raise ValueError("All three DataFrames must have the same shape.")
    if not (df1.columns.equals(df2.columns) and df1.columns.equals(df3.columns)):
        raise ValueError("All three DataFrames must have the same column labels.")

    n = len(df1)
    half_window = window_size // 2
    
    results = {}

    for col in df1.columns:
        x1 = df1[col].values
        x2 = df2[col].values
        x3 = df3[col].values
        
        sigma1_sq_local = np.zeros(n)
        sigma2_sq_local = np.zeros(n)
        sigma3_sq_local = np.zeros(n)
        
        # Slide a local window around each time index i
        for i in range(n):
            start = max(0, i - half_window)
            end = min(n, i + half_window + 1)
            
            diff12 = x1[start:end] - x2[start:end]
            diff13 = x1[start:end] - x3[start:end]
            diff23 = x2[start:end] - x3[start:end]
            
            if len(diff12) > 1:
                V12 = np.var(diff12, ddof=1)
                V13 = np.var(diff13, ddof=1)
                V23 = np.var(diff23, ddof=1)
            else:
                V12, V13, V23 = 0, 0, 0
            
            # Local TCH variance estimates
            sigma1_sq_local[i] = 0.5 * (V12 + V13 - V23)
            sigma2_sq_local[i] = 0.5 * (V12 + V23 - V13)
            sigma3_sq_local[i] = 0.5 * (V13 + V23 - V12)
        
        results[col] = (sigma1_sq_local, sigma2_sq_local, sigma3_sq_local)
    
    return results

    # Estimate local TCH variances
tch_results = estimate_local_tch_variances_multifeature(era2, gemb2, mar2, window_size=window_size)

def tch_results_to_dataframe(tch_results):
    
    data_dict = {}
    
    # Loop through each feature (e.g., 0, 1, ..., 11)
    for feature in tch_results:
        era, gemb, mar = tch_results[feature]
        data_dict[('era', feature)] = era
        data_dict[('gemb', feature)] = gemb
        data_dict[('mar', feature)] = mar

    # Create the DataFrame; assume all arrays are of the same length.
    df_result = pd.DataFrame(data_dict)
    
    df_result.columns = pd.MultiIndex.from_tuples(df_result.columns)
    
    return df_result

# Convert to a DataFrame:
df_tch = tch_results_to_dataframe(tch_results)

df_tch.to_csv('/home/df_tch.csv')

df_tch_gemb= df_tch['gemb']
df_tch_mar= df_tch['mar']
df_tch_era= df_tch['era']

df_tch_mar.to_csv('/home/mar_tch_data_unc.csv', index= False)
df_tch_gemb.to_csv('/home/gemb_tch_data_unc.csv', index= False)
df_tch_era.to_csv('/home/era_tch_data_unc.csv', index=False)